In [1]:
from collections import defaultdict
from tqdm.notebook import tqdm

In [2]:
data = open("input/15").read()

In [3]:
raw_grid, instructions = data.split("\n\n")

In [4]:
instructions = instructions.replace("\n", "")

In [5]:
grid = defaultdict(str)
grid2 = defaultdict(str)
my_pos = None
my_pos2 = None
for r, row in enumerate(raw_grid.split()):
    for c, elem in enumerate(row):
        if elem == "@":
            my_pos = (r, c)
            my_pos2 = (r, c * 2)
            elem = "."
        grid[(r, c)] = elem

        if elem == "O":
            elem1 = "["
            elem2 = "]"
        else:
            elem1 = elem2 = elem

        grid2[(r, c * 2)] = elem1
        grid2[(r, c * 2 + 1)] = elem2

In [6]:
instr_map = {
    ">": [0, 1],
    "<": [0, -1],
    "^": [-1, 0],
    "v": [1, 0],
}

# Part 1

In [7]:
def do_instr(instr, my_pos):
    d = instr_map[instr]
    wanted_pos = (my_pos[0] + d[0], my_pos[1] + d[1])
    elem = grid[wanted_pos]
    
    if elem == "#":
        return my_pos
        
    if elem == ".":
        return wanted_pos

    # TODO rremvoe
    if elem == "O":
        i = 1
        while True:
            next_pos = (my_pos[0] + d[0] * i, my_pos[1] + d[1] * i)
 
            elem = grid[next_pos]
            if elem == ".":
                grid[next_pos] = "O"
                grid[wanted_pos] = "."
                return wanted_pos
            if elem == "#":
                return my_pos

            i += 1

In [8]:
for instr in instructions:
    my_pos = do_instr(instr, my_pos)

In [9]:
def calculate_sum():
    price = 0
    for pos, val in grid.items():
        if val != "O":
            continue
        price += 100 * pos[0] + pos[1]
    return price

In [10]:
print(f"Answer #1: {calculate_sum()}")

Answer #1: 1514333


# Part 2

In [11]:
def do_instr2(instr, my_pos):
    d = instr_map[instr]
    wanted_pos = (my_pos[0] + d[0], my_pos[1] + d[1])
    elem = grid2[wanted_pos]
    
    if elem == "#":
        return my_pos
        
    if elem == ".":
        return wanted_pos
        
    if elem in ["[", "]"]:
        if instr in ["<", ">"]:
            # This is the "easy" one, left/right wont affect other than that row
            i = 1
            elems = []
            opposite = "[" if elem == "]" else "]"
            while True:
                next_pos = (my_pos[0] + d[0] * i, my_pos[1] + d[1] * i)
                n_elem = grid2[next_pos]
                elems.append([n_elem, next_pos])
                if n_elem == ".":
                    toggle = True
                    for e, pos in elems[1:]:
                        grid2[pos] = elem if toggle else opposite
                        toggle = not toggle
                    grid2[wanted_pos] = "."
                    return wanted_pos
                if n_elem == "#":
                    return my_pos
                i += 1
        else:
            return move_up_down(instr, my_pos)


In [12]:
def do_the_updates(instr, visited):
    pos_cache = {p: grid2[p] for p in visited}
    # Reset those positions to simplify update
    for p in visited:
        grid2[p] = "."
    if instr == "^":
        for pos in visited:
            above_pos = (pos[0] - 1, pos[1])
            grid2[above_pos] = pos_cache[pos]        
    else:
        for pos in visited:
            below_pos = (pos[0] + 1, pos[1])
            grid2[below_pos] = pos_cache[pos]
        pass
    

In [13]:
def move_up_down(instr, my_pos):
    d = instr_map[instr]
    wanted_pos = (my_pos[0] + d[0], my_pos[1] + d[1])
    elem = grid2[wanted_pos]
    queue = [wanted_pos]
    if elem == "]":
        queue.append((wanted_pos[0], wanted_pos[1] - 1))
    else:
        queue.append((wanted_pos[0], wanted_pos[1] + 1))
    visited = set()
    while queue:
        cur_pos = queue.pop()
        if cur_pos in visited:
            continue
        visited.add(cur_pos)
        grid_elem = grid2[cur_pos]
        if instr == "^":
            above_below_pos = (cur_pos[0] - 1, cur_pos[1])
        else:
            above_below_pos = (cur_pos[0] + 1, cur_pos[1])
        above_below_elem = grid2[above_below_pos]

        if above_below_elem == "#":
            # Won't work
            return my_pos

        if above_below_elem in ["]", "["]:
            queue.append(above_below_pos)
            if above_below_elem == grid_elem:
                continue
            else:
                if above_below_elem == "]":
                    queue.append((above_below_pos[0], above_below_pos[1] - 1))
                else:
                    queue.append((above_below_pos[0], above_below_pos[1] + 1))
    
    do_the_updates(instr, visited)
    return wanted_pos
                      

In [14]:
for instr in instructions:
    my_pos2 = do_instr2(instr, my_pos2)

In [15]:
def calculate_sum2():
    price = 0
    for pos, val in grid2.items():
        if val != "[":
            continue
        price += 100 * pos[0] + pos[1]
    return price

In [16]:
print(f"Answer #2: {calculate_sum2()}")

Answer #2: 1528453
